In [1]:
from utils import extract_all_features_for_count_threshold, extract_features_for_dist_threshold, get_custom_matrix_features
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')

In [ ]:

import pandas as pd
feature_list = ['']
threshold_mode = 'count'
num_nn = 4
threshold = 10
# colony 0
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]

colony0_features, f_list = extract_all_features_for_count_threshold(
    colony0_segmentation_path, num_nn=num_nn, threshold=threshold)
colony0_features['colony'] = [0 for i in range(len(colony0_features))]

colony0_matrix_features = get_custom_matrix_features(
    colony0_features, colony0_lineage_gt, f_list)

# colony 1 to 5
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)


# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
colony_candidate_features = [0, 0, 0, 0, 0, 0]
for i in [1, 2, 3, 4, 5]:
    print("finding features for colony {}".format(i))
    colony_candidate_features[i-1], _ = extract_all_features_for_count_threshold(os.path.join(
        path_to_colonies, 'colony00{}_segmentation.h5'.format(i)), num_nn=num_nn, threshold=threshold)
    colony_candidate_features[i-1]['colony'] = i
    # remove rows with time_id = 0
    colony_candidate_features[i-1] = colony_candidate_features[i -
                                                               1][colony_candidate_features[i-1].time_id != 0]
    # remove rows with bud_id == candid_id (this shouldn't happen in theory)
    colony_candidate_features[i-1] = colony_candidate_features[i-1][colony_candidate_features[i-1].bud_id !=
                                                                    colony_candidate_features[i-1].candid_id]


colonies_features = colony_candidate_features[0]
for i in [1, 2, 3, 4]:
    temp_colony = colony_candidate_features[i]
    colonies_features = pd.concat([colonies_features, temp_colony]).reset_index(drop=True)
colonies_features = colonies_features.reset_index(drop=True)

colonies_matrix_features = get_custom_matrix_features(colonies_features, colonies_gt, f_list)

all_candid_features = pd.concat([colonies_features, colony0_features]).reset_index(drop=True)
#save the features to a csv file
all_candid_features.to_csv(os.path.join(path_to_data, 'all_candid_features_{}_{}_nn_{}_threshold_19_features.csv'.format(threshold_mode, num_nn, threshold)))
all_matrix_features = pd.concat([colonies_matrix_features, colony0_matrix_features]).reset_index(drop=True)
all_gt = pd.concat([colonies_gt, colony0_lineage_gt]).reset_index(drop=True)

In [15]:
all_gt = pd.concat([colonies_gt, colony0_lineage_gt]).reset_index(drop=True)
all_gt.to_csv(os.path.join(path_to_data, 'all_gt.csv'))
all_gt

,parent_GT,bud_id,time_index,colony
0,-1,1,0,1
1,-1,2,0,1
2,2,3,4,1
3,1,4,7,1
4,4,5,27,1
...,...,...,...,...
867,91,364,146,0
868,132,365,146,0
869,111,366,146,0
870,224,367,146,0


In [17]:
from nn_model import BudDataset, LineageNN, train_nn, test_nn, cv_nn


In [20]:
# for the full list minus bad ones. with fillings (0.849886820007302 +/- 0.012254935663677358) (0.8559401241328952 +/- 0.009566047660644358)(0.8583570646221247 +/- 0.015580173489105039)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candid_features, all_gt, f_list, filling_features=[10,10,10,10,0,0,0,10,-1,-1,-1,-1,10,-1,-1,-1,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)


config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▂▃▄▄▄▄▅▅▅▅▅▅███████████
epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
eval_accuracy,▂▃▄▅▄▂▅▆▃▃▁▅▄█▄▄▇▂▄▄▅▆▄▄
lr,██▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁
patience,▁▁▁▁▂▂▃▁▂▂▃▄▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▅▆▆▆▇▇▇▇▇▇▇█▇▇█▇█████
best_accuracy,0.86061
epoch,23
eval_accuracy,0.82424
lr,0.00762
patience,10


early stopping at  34 LR:  0.006534999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▃▃▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▃▃▅▄▅▆▆▆▅▆▆▇▅▇▅▅▇▅▇▆▇▆█▆▆▇▂▆▇▆▅▅▅
lr,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▂▁▂▂▁▂▁▂▂▃▁▂▁▂▂▃▄▅▅▆▇▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇████
best_accuracy,0.85542
epoch,33
eval_accuracy,0.81928
lr,0.00663
patience,10


early stopping at  29 LR:  0.007029999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  16 LR:  0.008316999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▃▃▅▅███████████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval_accuracy,▁▃▃▅▂█▅▄▅▆▅▅█▃▅▅
lr,██▇▇▆▆▅▅▄▄▃▃▂▂▁▁
patience,▁▁▂▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▇▇▇▇▇▇█████
best_accuracy,0.85455
epoch,15
eval_accuracy,0.81818
lr,0.00842
patience,10


early stopping at  22 LR:  0.007722999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▂▂▃▃▃▅▅▅▅▅███████████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
eval_accuracy,▃▄▃▄▃▁▆▆▅▃▆█▅▅▆▄▄▇█▄▄▅
lr,██▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
patience,▁▁▂▁▂▂▁▂▂▃▄▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇██████
best_accuracy,0.86061
epoch,21
eval_accuracy,0.83636
lr,0.00782
patience,10


early stopping at  29 LR:  0.007029999999999997
accuracy:  0.8583570646221247 +/- 0.015580173489105039


In [22]:
# for the full list minus bad ones. without fillings (0.855947426067908 +/- 0.015363479495560519)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candid_features, all_gt, f_list)
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)


config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▃▆▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
eval_accuracy,▁▃▆▇▅▄▅▄▆▅▄▄▆█▇▅▆▆▆▆▅▅▂▅
lr,██▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁
patience,▁▁▁▁▂▂▃▄▅▅▆▇▇▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇████████
best_accuracy,0.85455
epoch,23
eval_accuracy,0.81212
lr,0.00762
patience,10


early stopping at  28 LR:  0.007128999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▁▁▁▅▅▅▅▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
eval_accuracy,▅▁▄▁▆▆▅▄█▇▅▇▆▅▇▇▇██▇██▆█▆██▇
lr,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
patience,▁▂▂▃▁▂▂▃▁▂▂▃▄▅▅▆▇▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████
best_accuracy,0.84337
epoch,27
eval_accuracy,0.83133
lr,0.00723
patience,10


early stopping at  29 LR:  0.007029999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  34 LR:  0.006534999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▁▁▁▄▄▄▄▄▄▄▆▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
eval_accuracy,▄▃▁▄▅▄▆▅▆▅▄▇▇▆▆▆▆▇▅▅▇▅▅█▇▅▇▇▇▇▆▅▇▆
lr,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▂▂▁▁▂▁▂▂▃▄▁▁▂▂▃▄▁▂▂▃▄▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
best_accuracy,0.86667
epoch,33
eval_accuracy,0.82424
lr,0.00663
patience,10


early stopping at  14 LR:  0.008514999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▄▄███████████
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
eval_accuracy,▃▅▄█▅▆▅▅▁▅█▇▇▅
lr,█▇▇▆▆▅▅▄▄▃▃▂▂▁
patience,▁▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▅▆▇▇▇▇▇████
best_accuracy,0.83636
epoch,13
eval_accuracy,0.8
lr,0.00861
patience,10


early stopping at  27 LR:  0.0072279999999999974
accuracy:  0.855947426067908 +/- 0.015363479495560519


# try normalizing before training to see if there is a improvement now that we have limited numbers every where


In [29]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

_19_f_list = ['dist_0','dist_max','dist_min','dist_std','poly_fit_budcm_candidcm','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']
all_candid_features_filtered = all_candid_features.loc[all_candid_features['dist_0']<11].reset_index(drop=True)
# Extract the subset of features to be normalized as a separate DataFrame
X = all_candid_features_filtered[_19_f_list]

# Apply min-max normalization to the subset of features
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

# Replace the original features with the normalized features in the dataframe
all_candid_features_filtered_normalized = all_candid_features_filtered.copy()
all_candid_features_filtered_normalized[_19_f_list] = X_norm

all_matrix_features_normalized = get_custom_matrix_features(all_candid_features_filtered_normalized, all_gt, _19_f_list)
all_matrix_features_normalized

,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
2,2,3,4,1,"[[0.09284766908852593, 0.009638973981007446, 0...","[2, -3, -3, -3]",0
3,1,4,7,1,"[[0.20761369963434992, 0.007620278021771029, 0...","[1, -3, -3, -3]",0
4,4,5,27,1,"[[0.2785430072655778, 0.010223675798476585, 0....","[4, -3, -3, -3]",0
5,1,6,27,1,"[[0.18569533817705186, 0.007620278021771029, 0...","[1, -3, -3, -3]",0
6,2,7,28,1,"[[0.18569533817705186, 0.006815783865651057, 0...","[2, 1, -3, -3]",0
...,...,...,...,...,...,...,...
856,90,353,145,0,"[[0.20761369963434992, 0.007620278021771029, 0...","[96, 227, 202, 90]",3
857,189,354,145,0,"[[0.20761369963434992, 0.007620278021771029, 0...","[142, 189, -3, -3]",1
858,221,355,145,0,"[[0.13130643285972257, 0.010776700527442208, 0...","[221, 55, 190, 337]",0
859,157,356,145,0,"[[0.09284766908852593, 0.006815783865651057, 0...","[157, 95, 294, 73]",0


In [33]:
all_candid_features_filtered_normalized

,bud_id,candid_id,time_id,dist_0,dist_max,dist_min,dist_std,poly_fit_budcm_candidcm,poly_fit_budcm_budpt,poly_fit_expansion_vector,...,position_bud_last,position_bud_first,orientation_bud_std,orientation_bud_max,orientation_bud_min,orientation_bud_last,orientation_bud_first,orientation_bud_last_minus_first,plyfit_orientation_bud,colony
0,3,2,4,0.092848,0.009639,0.092848,0.006449,0.093876,0.127785,0.106950,...,0.357290,0.297020,0.690556,0.876836,0.054801,0.729442,0.052915,0.861387,0.603746,1
1,4,1,7,0.207614,0.007620,0.092848,0.003885,0.096558,0.144178,0.129969,...,0.055665,0.070003,0.038510,0.971132,0.926270,0.972242,0.911225,0.539969,0.403292,1
2,5,4,27,0.278543,0.010224,0.207614,0.002557,0.085020,0.115990,0.097192,...,0.224003,0.176209,0.114991,0.844665,0.721803,0.732925,0.820036,0.462618,0.345158,1
3,6,1,27,0.185695,0.007620,0.092848,0.003956,0.089173,0.122027,0.097535,...,0.805050,0.916095,0.231401,0.975949,0.743845,0.976999,0.748149,0.627611,0.478145,1
4,7,2,28,0.185695,0.006816,0.185695,0.000000,0.088352,0.123391,0.103127,...,0.475448,0.446240,0.280480,0.333550,0.021105,0.259042,0.019728,0.633077,0.488255,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,356,157,145,0.092848,0.006816,0.000000,0.006754,0.099501,0.133475,0.095609,...,0.339222,0.314507,0.126752,0.129388,0.000000,0.104913,0.140288,0.489636,0.349220,0
2111,356,95,145,0.131306,0.004819,0.000000,0.005760,0.092948,0.134457,0.097720,...,0.183676,0.107145,0.858617,0.894451,0.000000,0.896513,0.691976,0.614915,0.594454,0
2112,356,294,145,0.207614,0.007620,0.000000,0.009107,0.076865,0.104054,0.086062,...,0.461684,0.444004,0.241880,0.275392,0.000000,0.101656,0.284478,0.412640,0.198500,0
2113,356,73,145,0.334767,0.012287,0.000000,0.012315,0.077603,0.115485,0.089137,...,0.173570,0.171440,0.958190,0.976801,0.000000,0.977840,0.805776,0.597958,0.561260,0


In [34]:
all_candid_features_normalized

,bud_id,candid_id,time_id,dist_0,dist_max,dist_min,dist_std,poly_fit_budcm_candidcm,poly_fit_budcm_budpt,poly_fit_expansion_vector,...,position_bud_last,position_bud_first,orientation_bud_std,orientation_bud_max,orientation_bud_min,orientation_bud_last,orientation_bud_first,orientation_bud_last_minus_first,plyfit_orientation_bud,colony
0,3,2,4,0.002544,0.007190,0.002556,0.004853,0.087489,0.097102,0.092832,...,0.357119,0.297020,0.691325,0.876836,0.054801,0.729442,0.053000,0.848520,0.609977,1
1,3,1,4,0.050874,0.054340,0.049625,0.004807,0.092563,0.105912,0.097172,...,0.774330,0.659055,0.124760,0.698800,0.543918,0.703294,0.611228,0.547894,0.418295,1
2,4,1,7,0.005688,0.005684,0.002556,0.002923,0.089250,0.108150,0.108056,...,0.055639,0.070003,0.040899,0.971132,0.926270,0.972242,0.911233,0.531904,0.412675,1
3,4,2,7,0.091750,0.091686,0.074515,0.015851,0.082546,0.105577,0.098785,...,0.376977,0.320081,0.143330,0.551005,0.375171,0.479372,0.368505,0.557578,0.433360,1
4,4,3,7,0.108070,0.107995,0.090363,0.016486,0.099976,0.123533,0.101175,...,0.341045,0.323936,0.141430,0.180655,0.006729,0.111614,0.005659,0.555068,0.431699,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,356,73,145,0.009171,0.009165,0.000000,0.009267,0.076798,0.088813,0.081051,...,0.173487,0.171440,0.958294,0.976801,0.000000,0.977840,0.805794,0.589026,0.568159,0
0,357,185,145,0.005688,0.005684,0.000000,0.005870,0.078608,0.086707,0.084339,...,0.145781,0.112898,0.273307,0.303229,0.000000,0.312638,0.043369,0.639074,0.666049,0
0,357,145,145,0.066575,0.084955,0.000000,0.092659,0.142596,0.154921,0.151948,...,0.940298,0.944137,0.859222,0.893218,0.000000,0.695340,0.894634,0.398007,0.194540,0
0,357,250,145,0.066769,0.068444,0.000000,0.081495,0.095368,0.105569,0.096892,...,0.139952,0.215056,0.704384,0.775805,0.000000,0.779341,0.474745,0.657225,0.701551,0


In [32]:
all_candid_features_normalized

,bud_id,candid_id,time_id,dist_0,dist_max,dist_min,dist_std,poly_fit_budcm_candidcm,poly_fit_budcm_budpt,poly_fit_expansion_vector,...,position_bud_last,position_bud_first,orientation_bud_std,orientation_bud_max,orientation_bud_min,orientation_bud_last,orientation_bud_first,orientation_bud_last_minus_first,plyfit_orientation_bud,colony
0,3,2,4,0.002544,0.007190,0.002556,0.004853,0.087489,0.097102,0.092832,...,0.357119,0.297020,0.691325,0.876836,0.054801,0.729442,0.053000,0.848520,0.609977,1
1,3,1,4,0.050874,0.054340,0.049625,0.004807,0.092563,0.105912,0.097172,...,0.774330,0.659055,0.124760,0.698800,0.543918,0.703294,0.611228,0.547894,0.418295,1
2,4,1,7,0.005688,0.005684,0.002556,0.002923,0.089250,0.108150,0.108056,...,0.055639,0.070003,0.040899,0.971132,0.926270,0.972242,0.911233,0.531904,0.412675,1
3,4,2,7,0.091750,0.091686,0.074515,0.015851,0.082546,0.105577,0.098785,...,0.376977,0.320081,0.143330,0.551005,0.375171,0.479372,0.368505,0.557578,0.433360,1
4,4,3,7,0.108070,0.107995,0.090363,0.016486,0.099976,0.123533,0.101175,...,0.341045,0.323936,0.141430,0.180655,0.006729,0.111614,0.005659,0.555068,0.431699,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,356,73,145,0.009171,0.009165,0.000000,0.009267,0.076798,0.088813,0.081051,...,0.173487,0.171440,0.958294,0.976801,0.000000,0.977840,0.805794,0.589026,0.568159,0
0,357,185,145,0.005688,0.005684,0.000000,0.005870,0.078608,0.086707,0.084339,...,0.145781,0.112898,0.273307,0.303229,0.000000,0.312638,0.043369,0.639074,0.666049,0
0,357,145,145,0.066575,0.084955,0.000000,0.092659,0.142596,0.154921,0.151948,...,0.940298,0.944137,0.859222,0.893218,0.000000,0.695340,0.894634,0.398007,0.194540,0
0,357,250,145,0.066769,0.068444,0.000000,0.081495,0.095368,0.105569,0.096892,...,0.139952,0.215056,0.704384,0.775805,0.000000,0.779341,0.474745,0.657225,0.701551,0


In [26]:
# for the normalized data with full list minus bad ones. with fillings (0.849886820007302 +/- 0.014954776486613947)(0.8425994888645493 +/- 0.012883609953286036)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candid_features_normalized, all_gt, f_list, filling_features=[10,10,10,10,0,0,0,10,-1,-1,-1,-1,10,-1,-1,-1,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▃▃▅▅▆▆▆▆▇▇▇▇▇▇███████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
eval_accuracy,▁▃▂▅▄▆▅▆▆▇▇▇▇▇▆█▇▆▇███▇▇▇▇▇▇█▇
lr,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
patience,▁▁▂▁▂▁▂▂▃▁▁▁▂▂▃▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇▇█████████████████
best_accuracy,0.84242
epoch,29
eval_accuracy,0.81818
lr,0.00703
patience,10


early stopping at  36 LR:  0.006336999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▄▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇███████████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▄▄▃▅▅▆▆▇▇▅▆▇▇▇▇▇███▇▇█▇▇█▇█▇█▇▇▇█▇▇
lr,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▂▂▁▁▁▂▁▂▂▃▄▅▅▆▇▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇███████████████████████
best_accuracy,0.83735
epoch,35
eval_accuracy,0.81928
lr,0.00644
patience,10


early stopping at  26 LR:  0.007326999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  26 LR:  0.007326999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▄▄▅▆▆▇▇▇▇▇███████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval_accuracy,▁▄▄▅▆▅▇▆▅▆▇█▆▇▆█▇▇▇▇▇▇▇▇█▇
lr,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
patience,▁▁▁▁▁▂▁▂▂▃▁▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▇▇▇▇▇▇███████████████
best_accuracy,0.82424
epoch,25
eval_accuracy,0.78788
lr,0.00743
patience,10


early stopping at  32 LR:  0.006732999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▅▅▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
eval_accuracy,▁▅▅▇▇▆▇▇▇▇▇▇▇▇▇█▇█▇▇▇█████▇█████
lr,███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
patience,▁▁▁▁▂▂▁▂▂▃▄▅▅▆▇▁▂▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▃▅▅▆▆▇▇▇▇▇▇▇███████████████████
best_accuracy,0.84242
epoch,31
eval_accuracy,0.8303
lr,0.00683
patience,10


early stopping at  23 LR:  0.007623999999999997
accuracy:  0.8389850310332237 +/- 0.008172180043855585


In [35]:
# for the filtered normalized data with full list minus bad ones. with fillings (0.842606790799562 +/- 0.01442356751359885)
f_list = ['dist_0','dist_std','poly_fit_budcm_budpt','poly_fit_expansion_vector','position_bud_std','position_bud_max','position_bud_min','position_bud_last','position_bud_first','orientation_bud_std','orientation_bud_max','orientation_bud_min','orientation_bud_last','orientation_bud_first','orientation_bud_last_minus_first','plyfit_orientation_bud']

matrix_feat_custom = get_custom_matrix_features(all_candid_features_normalized, all_gt, f_list, filling_features=[10,10,10,10,0,0,0,10,-1,-1,-1,-1,10,-1,-1,-1,-1,10,-1])
config = {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [len(f_list)*4, 64, 5], 'augment': True , 'features':f_list,'train_set': 'all',}
models, accuracies = cv_nn(matrix_feat_custom, config=config)

config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 1}
yes. I am reloaded


best_accuracy,▁▅▅▆▇▇█████████████████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
eval_accuracy,▁▅▄▆▇▇█▆▇▇▇██▇▇█████▇▇█
lr,██▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁
patience,▁▁▂▁▁▂▁▂▂▃▄▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇███████████
best_accuracy,0.84242
epoch,22
eval_accuracy,0.8303
lr,0.00772
patience,10


early stopping at  29 LR:  0.007029999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 2}
yes. I am reloaded


best_accuracy,▁▂▂▅▅▆▆▆▆▇▇▇█████████████████
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
eval_accuracy,▁▂▂▅▅▆▅▅▆▇▇▆█▇▇▇▇▆█▇▇█▇▇▇▇▇▇█
lr,██▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁
patience,▁▁▂▁▂▁▂▂▁▁▂▂▁▂▂▃▄▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇█▇██████████████
best_accuracy,0.8494
epoch,28
eval_accuracy,0.84337
lr,0.00713
patience,10


early stopping at  23 LR:  0.007623999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 3}
yes. I am reloaded


...

early stopping at  26 LR:  0.007326999999999998
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 4}
yes. I am reloaded


best_accuracy,▁▄▅▆▆▇▇▇▇▇████████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval_accuracy,▁▄▅▆▆▇▇▇▇▇█▇██▆█▇▇█▇▇▇█▆██
lr,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
patience,▁▁▁▁▂▁▁▂▁▂▁▂▂▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▇▇▇▇█████████████████
best_accuracy,0.81818
epoch,25
eval_accuracy,0.81818
lr,0.00743
patience,10


early stopping at  36 LR:  0.006336999999999997
config:  {'epoch_n': 200, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [64, 64, 5], 'augment': True, 'features': ['dist_0', 'dist_std', 'poly_fit_budcm_budpt', 'poly_fit_expansion_vector', 'position_bud_std', 'position_bud_max', 'position_bud_min', 'position_bud_last', 'position_bud_first', 'orientation_bud_std', 'orientation_bud_max', 'orientation_bud_min', 'orientation_bud_last', 'orientation_bud_first', 'orientation_bud_last_minus_first', 'plyfit_orientation_bud'], 'train_set': 'all', 'cv_number': 5}
yes. I am reloaded


best_accuracy,▁▄▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▄▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██▇█▇██▇▇███
lr,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
patience,▁▁▂▁▁▁▂▁▂▂▁▂▂▃▄▅▅▆▇▇█▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇███████████████████████
best_accuracy,0.84848
epoch,35
eval_accuracy,0.84848
lr,0.00644
patience,10


early stopping at  29 LR:  0.007029999999999997
accuracy:  0.842606790799562 +/- 0.01442356751359885
